In [1]:
library("AzureML")
ws <- workspace()
dat <- download.datasets(ws, "FinalD.csv")

In [2]:
names(dat) = c("Y","Dl","Nlp","Cp")
dat = data.frame(dat)
head(dat)

Y,Dl,Nlp,Cp
Buy,1.00,1,P
Buy,0.75,1,P
Buy,0.50,1,P
Buy,0.25,1,P
Buy,0.00,1,P
Buy,-0.25,1,P


In [3]:
#print(dat$Y[38])

In [4]:
for(i in 1:length(dat$Y)) # {"sell":1,"Hold":2,"Buy":3}
    for(j in length(dat$Y))
        if(dat$Y[i] == "Sell")
           { dat$Y[i] = 1
        }else if(dat$Y[i] == "Hold")
            {dat$Y[i] = 2
        }else{
            dat$Y[i] = 3 
        }
            
for(i in 1:length(dat$Cp)) ## {"Cp$Profit": 1, "Cp$loss":-1,"Cp$cost":0}
    for(j in length(dat$Cp))
        if(dat$Cp[i] == "P")
           { dat$Cp[i] = 1
        }else if(dat$Cp[i] == "L")
            {dat$Cp[i] = -1
        }else{
            dat$Cp[i] = 0 
        }        

In [5]:
#print(dat$Y[38])
#ex_dat = dat

In [6]:
#print(dat$Cp[112])

In [7]:
mul = lm(Y ~ Nlp+Dl, data = dat)

# Linear Regression for continous variables

In [8]:
print(mul)


Call:
lm(formula = Y ~ Nlp + Dl, data = dat)

Coefficients:
(Intercept)          Nlp           Dl  
     1.7778       0.4667       0.3012  



# Decision Tree

In [10]:
install.packages('rpart', repos='http://cran.us.r-project.org')

Installing package into '/home/nbcommon/R'
(as 'lib' is unspecified)



The downloaded source packages are in
	'/tmp/Rtmp85q35R/downloaded_packages'


In [11]:
library(rpart)

# Shuffle the dataset

In [12]:
n <- nrow(dat)
dfs <- dat[sample(n),]

# Split the data in train and test


In [13]:
train_indices <- 1:round(0.7 * n)
train <- dfs[train_indices, ]
test_indices <- (round(0.7 * n) + 1):n
test <- dfs[test_indices, ]

In [14]:
install.packages('rpart',repos='http://cran.us.r-project.org')
library(rpart)

Installing package into '/home/nbcommon/R'
(as 'lib' is unspecified)



The downloaded source packages are in
	'/tmp/Rtmp85q35R/downloaded_packages'


In [15]:
tree <- rpart(Y ~., train, method = "class", minsplit=2, minbucket=1)

In [16]:
library(RColorBrewer)

In [17]:
pred_test <- predict(tree, test, type = "class")
conf <- table(test$Y, pred_test)
acc <- sum(diag(conf))/sum(conf)

In [18]:
acc

[1] 0.7534247

# Multinomial Logistic Regression 

In [19]:
install.packages('foreign', repos='http://cran.us.r-project.org')

Installing package into '/home/nbcommon/R'
(as 'lib' is unspecified)



The downloaded source packages are in
	'/tmp/Rtmp85q35R/downloaded_packages'


In [20]:
library(foreign)

In [21]:
head(dat)

Y,Dl,Nlp,Cp
3,1.00,1,1
3,0.75,1,1
3,0.50,1,1
3,0.25,1,1
3,0.00,1,1
3,-0.25,1,1


In [22]:
library(nnet)

In [23]:
#p = dat$Nlp[sample(nrow(dat), 50), ]
#q = dat[, sample(ncol(df), 1)
q = dat$Nlp

In [24]:
test = multinom(Y ~ Dl + Nlp + Cp, data = dat)

# weights:  18 (10 variable)
initial  value 266.962786 
iter  10 value 151.907507
final  value 150.685778 
converged


In [25]:
summary(test)
test$softmax

Call:
multinom(formula = Y ~ Dl + Nlp + Cp, data = dat)

Coefficients:
  (Intercept)         Dl      Nlp       Cp0       Cp1
2   2.9782637 -0.1345859 0.720734 -4.507300 -4.187640
3  -0.2755069  1.9108606 2.669195 -1.633686 -1.124316

Std. Errors:
  (Intercept)        Dl       Nlp       Cp0       Cp1
2   0.5270166 0.3366646 0.3428914 0.6276486 0.6160048
3   0.7492405 0.4069105 0.4458615 0.7819000 0.7756301

Residual Deviance: 301.3716 
AIC: 321.3716 

[1] TRUE

In [26]:
predict(test,dat)#{"sell":1,"Hold":2,"Buy":3}

[1] 3 3 3 3 3 3 3 1 1 3 3 3 3 3 3 1 1 1 3 3 3 3 1 1 1 1 1 3 3 3 1 1 1 1 1 1 3
 [38] 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [75] 1 1 1 1 1 1 1 3 3 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
[112] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
[149] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 1 1 1 3 3 3 3 3 1 1 1 1 3 3 3 1 1
[186] 1 1 1 1 3 3 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[223] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
Levels: 1 2 3

In [27]:
Z = summary(test)$coefficients/summary(test)$standard.errors

In [28]:
head(fitted(test))

,1,2,3
1,0.03907995,0.02095508,0.9399650
2,0.06070980,0.03366717,0.9056230
3,0.09237710,0.05298154,0.8546414
4,0.13640650,0.08091104,0.7826825
5,0.19334590,0.11860969,0.6880444
6,0.26031486,0.16515687,0.5745283


In [29]:
dat = as.vector(dat)
head(dat)
#set.seed(50)
#dat1 = sample(dat,size = 50,replace = T)

Y,Dl,Nlp,Cp
3,1.00,1,1
3,0.75,1,1
3,0.50,1,1
3,0.25,1,1
3,0.00,1,1
3,-0.25,1,1


In [30]:
dat1 = dat[sample(nrow(dat),50), ]
head(dat1)

,Y,Dl,Nlp,Cp
243,1,-1.00,-1.00,0
170,1,-0.75,1.00,0
162,1,-1.00,-1.00,-1
131,2,0.00,-0.25,-1
218,1,0.75,-0.50,0
8,2,-0.75,1.00,1


In [31]:
head(dat1)

,Y,Dl,Nlp,Cp
243,1,-1.00,-1.00,0
170,1,-0.75,1.00,0
162,1,-1.00,-1.00,-1
131,2,0.00,-0.25,-1
218,1,0.75,-0.50,0
8,2,-0.75,1.00,1


In [32]:
predicted = predict(test,dat1,type="probs")

In [33]:
head(predicted)#{"sell":1,"Hold":2,"Buy":3}

,1,2,3
243,0.89116176,0.1074839,0.0013542896
170,0.49923991,0.2460965,0.2546635659
162,0.08372130,0.9156269,0.0006517663
131,0.05617145,0.9219479,0.0218806539
218,0.76911413,0.1050984,0.1257875169
8,0.39564028,0.2684869,0.3358727969


In [34]:
bpp = cbind(dat1,predicted)# {"Cp$Profit": 1, "Cp$loss":-1,"Cp$cost":0}

In [35]:
bpp

,Y,Dl,Nlp,Cp,1,2,3
243,1,-1.00,-1.00,0,0.89116176,0.10748395,0.0013542896
170,1,-0.75,1.00,0,0.49923991,0.24609653,0.2546635659
162,1,-1.00,-1.00,-1,0.08372130,0.91562693,0.0006517663
131,2,0.00,-0.25,-1,0.05617145,0.92194790,0.0218806539
218,1,0.75,-0.50,0,0.76911413,0.10509835,0.1257875169
8,2,-0.75,1.00,1,0.39564028,0.26848693,0.3358727969
6,3,-0.25,1.00,1,0.26031486,0.16515687,0.5745282687
40,3,0.25,0.00,1,0.59305271,0.17110205,0.2358452394
20,3,0.75,0.50,1,0.18818442,0.07278234,0.7390332318
59,1,0.00,-0.50,1,0.78552734,0.16346708,0.0510055817
